In [1]:
import onnx
import onnxruntime as ort
from onnxruntime.transformers import optimizer
# 创建ONNX模型
model = onnx.ModelProto()
graph = model.graph
graph.name = "example"
x = graph.add_input(name="x", type="float", shape=[1, 2])
cond = graph.add_input(name="cond", type="bool", shape=[1])
y_init = graph.add_input(name="y_init", type="float", shape=[1, 2])
z_init = graph.add_input(name="z_init", type="float", shape=[1, 2])
y = graph.add_node(op_type="Add", inputs=[x, y_init], outputs=["y"])
z = graph.add_node(op_type="Sub", inputs=[x, z_init], outputs=["z"])
if_node = graph.add_node(op_type="If", inputs=[cond])
if_node.add_output("y_out")
if_node.add_output("z_out")
then_branch = if_node.add_attribute("then_branch")
else_branch = if_node.add_attribute("else_branch")
# 添加then分支
then_graph = then_branch.graph
then_x = then_graph.add_input(name="x", type="float", shape=[1, 2])
then_y = then_graph.add_node(op_type="Mul", inputs=[then_x, y], outputs=["y"])
then_branch.add_output("y_out", then_y.output[0])
# 添加else分支
else_graph = else_branch.graph
else_x = else_graph.add_input(name="x", type="float", shape=[1, 2])
else_z = else_graph.add_node(op_type="Mul", inputs=[else_x, z], outputs=["z"])
else_branch.add_output("z_out", else_z.output[0])
# 添加输出节点
output = graph.add_node(op_type="Add", inputs=[if_node.output[0], if_node.output[1]], outputs=["output"])
graph.add_output(name="output", type="float", shape=[1, 2])
# 创建ONNX Runtime会话
session = ort.InferenceSession(model.SerializeToString())
# 运行模型
input_names = [input.name for input in session.get_inputs()]
output_names = [output.name for output in session.get_outputs()]
input_data = {"x": np.array([[1.0, 2.0]])}
input_data["cond"] = np.array([True])
input_data["y_init"] = np.array([[3.0, 4.0]])
input_data["z_init"] = np.array([[5.0, 6.0]])
output = session.run(output_names, input_data)
# 处理输出数据

AttributeError: add_input